In [1]:
from sklearn.metrics import (
    root_mean_squared_error,  # type: ignore
    mean_absolute_error,
    mean_absolute_percentage_error,
)
import numpy as np
import pandas as pd
import torch
from chronos import ChronosPipeline
import plotly.graph_objects as go
import plotly.express as px

In [2]:
df_training = pd.read_parquet("../../data/time_series/train_df.parquet")
df_validation = pd.read_parquet("../../data/time_series/validation_df.parquet")
df = pd.concat([df_training, df_validation])
gap_training = df["gap"]

In [3]:
df_test = pd.read_parquet("../../data/time_series/test_df.parquet")

In [4]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

/home/david/eafit/proyecto-integrador-semestre-2/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



## Forecasting


In [5]:
gap_test = df_test["gap"]

In [6]:
# context must be either a 1D tensor, a list of 1D tensors,
# or a left-padded 2D tensor with batch as the first dimension
context = torch.tensor(gap_training)
prediction_length = len(gap_test)
forecast = pipeline.predict(
    context, prediction_length
)  # shape [num_series, num_samples, prediction_length]

/tmp/ipykernel_3537369/3288026477.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [7]:
# visualize the forecast
forecast_index = range(len(gap_training), len(gap_training) + prediction_length)
low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)
test_index = df_test.index

## Métricas

Se calcula:

- RMSE (Root Mean Squared Error)
- MAE (Mean Absolute Error)
- MAPE (Mean Absolute Percentage Error)


In [8]:
rmse = root_mean_squared_error(gap_test, median)
mae = mean_absolute_error(gap_test, median)
mape = mean_absolute_percentage_error(gap_test, median)
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}")

RMSE: 13866.63463378281
MAE: 11808.748372395834
MAPE: 0.14153724137807647


## Visualización


In [11]:
data = {
    "Fecha": gap_test.index,
    "Valor real": gap_test,
    "Predicción": median,
}

# Convertir a DataFrame de pandas
plot_df = pd.DataFrame(data)

# Crear la figura usando Plotly Express
fig = px.line(
    plot_df,
    x="Fecha",
    y=["Valor real", "Predicción"],
    labels={"value": "Gap", "variable": "Legend"},
    title="Test vs Predicciones usando Chronos T5 Small",
    markers=True,
)

# Mostrar la figura
fig.show()